## Load the data
## Clean the data
## Modelling

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('./data.csv')

In [3]:
X,y = data.drop('target',axis=1),data['target']

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25)

In [6]:
import torch
import torch.nn as nn

In [7]:
import numpy as np

In [8]:
X_train = torch.from_numpy(np.array(X_train).astype(np.float32))
y_train = torch.from_numpy(np.array(y_train).astype(np.float32))
X_test = torch.from_numpy(np.array(X_test).astype(np.float32))
y_test = torch.from_numpy(np.array(y_test).astype(np.float32))

In [9]:
X_train.shape

torch.Size([227, 13])

In [10]:
X_test.shape

torch.Size([76, 13])

In [11]:
y_train.shape

torch.Size([227])

In [12]:
y_test.shape

torch.Size([76])

#### Modelling

In [13]:
import torch.nn.functional as F

In [14]:
class Test_Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(13,64)
        self.fc2 = nn.Linear(64,128)
        self.fc3 = nn.Linear(128,256)
        self.fc4 = nn.Linear(256,512)
        self.fc5 = nn.Linear(512,1024)
        self.fc6 = nn.Linear(1024,512)
        self.fc7 = nn.Linear(512,1)
    
    def forward(self,X):
        preds = self.fc1(X)
        preds = F.relu(preds)
        preds = self.fc2(preds)
        preds = F.relu(preds)
        preds = self.fc3(preds)
        preds = F.relu(preds)
        preds = self.fc4(preds)
        preds = F.relu(preds)
        preds = self.fc5(preds)
        preds = F.relu(preds)
        preds = self.fc6(preds)
        preds = F.relu(preds)
        preds = self.fc7(preds)
        return F.sigmoid(preds)

In [15]:
device = torch.device('cuda')

In [16]:
X_train = X_train.to(device)
y_train = y_train.to(device)
X_test = X_test.to(device)
y_test = y_test.to(device)

In [17]:
PROJECT_NAME = 'Heart-Disease-UCI'

In [18]:
def get_loss(criterion,X,y,model):
    model.eval()
    with torch.no_grad():
        preds = model(X.float().to(device))
        preds = preds.to(device)
        y = y.to(device)
        loss = criterion(preds,y)
    model.train()
    return loss.item()
def get_accuracy(X,y,model):
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for i in range(len(X)):
            pred = model(X[i].float().to(device))
            pred.to(device)
            if round(int(pred[0])) == round(int(y[i])):
                correct += 1
            total += 1
        if correct == 0:
            correct += 1
    model.train()
    return round(correct/total,3)

In [19]:
import wandb

In [20]:
from tqdm import tqdm

In [94]:
EPOCHS = 51

In [95]:
model = Test_Model().to(device)
optimizer = torch.optim.SGD(model.parameters(),lr=0.25)
criterion = nn.L1Loss()
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(EPOCHS)):
    preds = model(X_train.float().to(device))
    preds = preds.view(len(preds),)
    preds.to(device)
    loss = criterion(preds,y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    wandb.log({'loss':loss.item(),'val_loss':get_loss(criterion,X_test,y_test,model),'accuracy':get_accuracy(X_train,y_train,model),'val_accuracy':get_accuracy(X_test,y_test,model)})
wandb.finish()

100%|██████████| 237/237 [00:22<00:00, 10.54it/s]


loss,0.41698
val_loss,0.48467
accuracy,0.471
val_accuracy,0.408
_runtime,29
_timestamp,1621447383
_step,236


loss,▆▆▆▆▆▆▆▅▆▆█▆▆▆▆▅▃▆▂▆▃▃▂▄▅▆▄▄▂▃▂▆▆▆▅▃▂▃▁▁
val_loss,▂▃▁▁▁▁▃▇▃▃█▁▁▁▂▆▁▁▃▂▆▇█▂▂▁▂▃▅▇▅▁▁▁▂▆▂▆▁▇
accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▇▇▁▁▁▁▇▁
val_accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁█▇▅▁▁▁▁▆▁
_runtime,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


In [96]:
preds

tensor([5.5819e-04, 2.8326e-04, 6.2539e-05, 9.8949e-01, 4.3737e-04, 2.8420e-05,
        9.7961e-01, 1.4418e-04, 4.7761e-08, 1.6036e-04, 3.1676e-01, 9.8086e-01,
        1.2128e-05, 8.3767e-03, 1.9991e-05, 2.5457e-03, 9.6015e-05, 5.8040e-05,
        9.9570e-01, 1.3960e-05, 3.7197e-04, 7.3834e-05, 5.8934e-05, 9.6447e-01,
        3.1743e-04, 3.7959e-02, 9.6696e-05, 9.6502e-01, 1.0298e-03, 2.9507e-04,
        4.0898e-07, 8.8783e-01, 3.3252e-06, 1.6466e-04, 3.9444e-07, 1.2917e-04,
        9.3437e-05, 9.9460e-01, 1.8802e-08, 3.1438e-07, 3.5796e-04, 3.5255e-04,
        4.9175e-08, 2.1855e-05, 3.0600e-04, 5.9947e-06, 2.5248e-05, 6.3497e-04,
        1.2389e-04, 9.3540e-01, 6.0892e-07, 9.4782e-01, 1.7383e-06, 4.5896e-03,
        8.3873e-08, 9.4530e-01, 1.5831e-05, 2.6439e-03, 4.1464e-05, 9.6853e-05,
        1.1988e-02, 1.6419e-06, 4.7123e-06, 1.0156e-04, 2.8734e-02, 9.9041e-03,
        4.1818e-03, 8.1438e-01, 6.6344e-08, 2.0346e-05, 7.2938e-04, 2.5736e-07,
        5.5266e-05, 9.9882e-01, 6.5599e-

In [97]:
torch.round(preds)[:25]

tensor([0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 0., 0., 1., 0.], device='cuda:0', grad_fn=<SliceBackward>)

In [98]:
y_train[:25]

tensor([1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 1., 0.,
        1., 1., 0., 0., 0., 0., 1.], device='cuda:0')